## Import python file for pre-processing events data files

In [1]:
from process_data import *

In [2]:
import cassandra

## Create a connection to the database

In [3]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect()
    session.execute("Create keyspace if not exists sparkify with replication= \
    {'class': 'SimpleStrategy', 'replication_factor': 1};")
except Exception as e:
    print(e)
    

## Connect to sparkify keyspace

In [4]:
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

## Walk through the events directory and get a list of all the files

In [5]:
#Return a list of absolute paths of the csv files in the event data directory
files_list = get_file_paths('event_data')

# Pull all the contents of the csv files into a new file

In [6]:
#Create csv file and write the header rows
write_file()

In [7]:
#Write the contents from the list of files to the new csv file
read_write_file(files_list)

The new csv file titled `new_events_data.csv` contains the following columns:
* artist 
* first_name
* gender
* itemInSession
* last_name
* length
* level(free or paid)
* location
* sessionId
* song_title
* userId

## Modeling the data for Apache Cassandra

Since Apache Cassandra models the database tables on the queries, I will model the database tables against these 3 queries:
1. Give me the artist, song title, and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user(first and last name) for userId = 10, sessionId = 182
3. Give me every username(first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Create table for the first query

In [8]:
session.execute("CREATE TABLE IF NOT EXISTS songs (sessionId INT, itemInSession INT, artist TEXT, \
song_title TEXT, song_length FLOAT, PRIMARY KEY(sessionId, itemInSession)) ")

In [9]:
#load the contents of the new csv file into the database
query = "INSERT INTO songs (sessionId, itemInSession, artist, song_title, song_length) \
    VALUES (%s, %s, %s, %s, %s)"
with open("new_events_data.csv", "r") as csvfile:
    csv_reader = csv.reader(csvfile)
    #skip the header row
    next(csv_reader)
    #Iterate through the results and create cql insert statement
    for lines in csv_reader:
        session.execute(query, (int(lines[8]), int(lines[3]), lines[0], \
                                lines[9], float(lines[5])))
            

### Run  the first query to verify the data is inserted correctly

In [10]:
rows = session.execute("SELECT artist, song_title, song_length FROM songs WHERE sessionId = 338 \
AND itemInSession = 4")
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Create table for the second query

In [11]:
session.execute("CREATE TABLE IF NOT EXISTS artists (userId INT, sessionId INT, artist TEXT, \
song_title TEXT, first_name TEXT, last_name TEXT, itemInSession INT, level TEXT, \
PRIMARY KEY((userId, sessionId), itemInSession))")

In [12]:
#Load data into the artists table
query = "INSERT INTO artists (userId, sessionId, artist, song_title, \
first_name, last_name, itemInSession, level) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
with open("new_events_data.csv", "r") as csvfile:
    csv_reader = csv.reader(csvfile)
    #skip the header row
    next(csv_reader)
    #Iterate through the results and create cql insert statement
    for lines in csv_reader:
        if lines[10] == 'nan':
            lines[10] = int(0)
        else:
            lines[10] = int(float(lines[10]))
        session.execute(query, (lines[10], int(lines[8]), lines[0], lines[9], \
                                lines[1], lines[4], int(lines[3]), lines[6]))

### Run a select statement to verify the data is inserted correctly

In [13]:
rows = session.execute("SELECT artist, song_title, first_name, last_name FROM artists \
WHERE userId = 10 AND sessionId = 182")
for row in rows:
    #print(row.song_title)
    print(row.artist, "|", row.song_title, "|", row.first_name, "|", row.last_name)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


### Create table for the third query

In [14]:
session.execute("CREATE TABLE IF NOT EXISTS users(song_title TEXT, first_name TEXT, \
last_name TEXT, gender TEXT, PRIMARY KEY(song_title, first_name, last_name))")

In [15]:
#load the contents of the new csv file into the users table
query = "INSERT INTO users (song_title, first_name, last_name, gender) \
    VALUES (%s, %s, %s, %s)"
with open("new_events_data.csv", "r") as csvfile:
    csv_reader = csv.reader(csvfile)
    #skip the header row
    next(csv_reader)
    #Iterate through the results and create cql insert statement
    for lines in csv_reader:
        session.execute(query, (lines[9], lines[1], lines[4], lines[2]))

### Run the query to verify the data is inserted correctly

In [16]:
rows = session.execute("SELECT first_name, last_name FROM users WHERE \
song_title = 'All Hands Against His Own'")
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Sara Johnson
Tegan Levine


## Drop the tables


In [17]:
session.execute("DROP TABLE songs")
session.execute("DROP TABLE users")
session.execute("DROP TABLE artists")

## Close the keyspace and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()